In [1]:
# add your name here. Eg: yourname = 'vult5'
yourname = 'vult5'

# Example 1 - Simple pipeline

In [2]:
import kfp
from kfp import components, dsl
from easykubeflow import Utils

In [3]:
client = kfp.Client()
Utils(client)

In [4]:
experiment=f'{yourname}-Examples'
run='Dummy'

In [5]:
client.create_experiment(experiment)

{'created_at': datetime.datetime(2020, 8, 11, 4, 5, 39, tzinfo=tzutc()),
 'description': None,
 'id': '49772a1f-2bc5-4593-90c7-2034a01577b1',
 'name': 'vult5-Examples',
 'resource_references': None,
 'storage_state': None}

In [6]:
def print_op(message):
    return dsl.ContainerOp(
        name='Print message',
        image='asia.gcr.io/vinid-data-science-np/vu-example/base:1.0.0',
        command=['sh', '-c'],
        arguments=[f'echo {message}']
    )

In [7]:
@dsl.pipeline(
    name='Example',
    description='Dummy example pipeline'
)
def pipeline(message):
    print_op(message)

In [8]:
run_result = client.create_run_from_pipeline_func(pipeline, 
                                                  arguments={'message':'This is example message'},
                                                  run_name=run,
                                                  experiment_name=experiment,
                                                  namespace='vult5')
print(run_result)

RunPipelineResult(run_id=7525ae89-78df-4e2b-b4c3-31b3f899350c)


In [9]:
client.runs.delete_run(run_result.run_id)

{}

# Example 2 - Serial

In [10]:
import kfp
from kfp import components, dsl
from easykubeflow import Utils

In [11]:
client = kfp.Client()
Utils(client)

In [12]:
experiment=f'{yourname}-Examples'
run='Serial'

In [13]:
client.create_experiment(experiment)

{'created_at': datetime.datetime(2020, 8, 11, 4, 5, 39, tzinfo=tzutc()),
 'description': None,
 'id': '49772a1f-2bc5-4593-90c7-2034a01577b1',
 'name': 'vult5-Examples',
 'resource_references': None,
 'storage_state': None}

In [14]:
def random(low, high):
    return dsl.ContainerOp(
        name='Generate random number between 0 and 1',
        image='asia.gcr.io/vinid-data-science-np/vu-example/base:1.0.0',
        command=['sh', '-c'],
        arguments=[f"""python3 -c "import random; result = random.randint({low},{high}); print(result)" | tee /tmp/output"""],
        file_outputs={'output': '/tmp/output'}
    )

In [15]:
def print_op(message):
    return dsl.ContainerOp(
        name='Print message',
        image='asia.gcr.io/vinid-data-science-np/vu-example/base:1.0.0',
        command=['sh', '-c'],
        arguments=[f'echo {message}']
    )

In [16]:
@dsl.pipeline(
    name='Example',
    description='Dummy example pipeline'
)
def serial():
    random_task = random(0,9)
    print_op(random_task.output)
    random_task1 = random(10,19).after(random_task)
    print_op(random_task1.output)

In [18]:
run_result = client.create_run_from_pipeline_func(serial, 
                                                  arguments={},
                                                  run_name=run,
                                                  experiment_name=experiment,
                                                  namespace='vult5')

In [19]:
client.runs.delete_run(run_result.run_id)

{}

# Example 3 - Condition

In [20]:
import kfp
from kfp import components, dsl
from easykubeflow import Utils

In [21]:
client = kfp.Client()
Utils(client)

In [22]:
experiment=f'{yourname}-Examples'
run='flipcoin'

In [23]:
def random():
    return dsl.ContainerOp(
        name='Generate random number between 0 and 1',
        image='asia.gcr.io/vinid-data-science-np/vu-example/base:1.0.0',
        command=['sh', '-c'],
        arguments=["""python3 -c "import random; result = 'heads' if random.randint(0,1) == 0 else 'tails'; print(result)" | tee /tmp/output"""],
        file_outputs={'output': '/tmp/output'}
    )

In [24]:
def print_op(message):
    return dsl.ContainerOp(
        name='Print message',
        image='asia.gcr.io/vinid-data-science-np/vu-example/base:1.0.0',
        command=['sh', '-c'],
        arguments=[f'echo {message}']
    )

In [25]:
@dsl.pipeline(
    name='Example',
    description='Dummy example pipeline'
)
def flip_coin():
    flip = random()
    with dsl.Condition(flip.output == 'heads'):
        print_op('Result is heads')
    with dsl.Condition(flip.output == 'tails'):
        print_op('Result is tails')

In [26]:
run_result = client.create_run_from_pipeline_func(flip_coin, 
                                                  arguments={},
                                                  run_name=run,
                                                  experiment_name=experiment,
                                                  namespace='vult5')

In [27]:
client.runs.delete_run(run_result.run_id)

{}

# Example 4 - Loop

In [29]:
import kfp
from kfp import components, dsl
from easykubeflow import Utils

In [30]:
client = kfp.Client()
Utils(client)

In [31]:
experiment=f'{yourname}-Examples'
run='loop'

In [32]:
def random(low, high):
    return dsl.ContainerOp(
        name='Generate random number between 0 and 1',
        image='asia.gcr.io/vinid-data-science-np/vu-example/base:1.0.0',
        command=['sh', '-c'],
        arguments=[f"""python3 -c "import random; result = random.randint({low},{high}); print(result)" | tee /tmp/output"""],
        file_outputs={'output': '/tmp/output'}
    )

In [33]:
def print_op(message):
    return dsl.ContainerOp(
        name='Print message',
        image='asia.gcr.io/vinid-data-science-np/vu-example/base:1.0.0',
        command=['sh', '-c'],
        arguments=[f'echo {message}']
    )

In [34]:
@dsl.pipeline(
    name='Example',
    description='Dummy example pipeline'
)
def loop(lst:list):
    random0 = random(0, 10)
    with dsl.ParallelFor(lst) as item:
        random1 = random(0, item).after(random0)
        print_op(random1.output)
    

In [35]:
run_result = client.create_run_from_pipeline_func(loop, 
                                                  arguments={'lst':[5,4,2,6,9]},
                                                  run_name=run,
                                                  experiment_name=experiment,
                                                  namespace='vult5')

In [36]:
client.runs.delete_run(run_result.run_id)

{}

# Example 5 - Upload pipeline

In [40]:
import kfp
from kfp import components, dsl
from easykubeflow import Utils

In [41]:
client = kfp.Client()
Utils(client)

In [43]:
experiment=f'{yourname}-Examples'
run='upload_pipeline'

In [44]:
def print_op(message):
    return dsl.ContainerOp(
        name='Print message',
        image='asia.gcr.io/vinid-data-science-np/vu-example/base:1.0.0',
        command=['sh', '-c'],
        arguments=[f'echo {message}']
    )

In [47]:
@dsl.pipeline(
    name='Example',
    description='Dummy example pipeline'
)
def upload_pipeline(lst:list):
    echo0 = print_op('task0')
    echo1 = print_op('task1')
    echo1.after(echo0)

In [48]:
upload_pipeline.__name__

'upload_pipeline'

In [49]:
from kfp.compiler import Compiler
package_name = upload_pipeline.__name__ + '.yaml'
Compiler().compile(upload_pipeline, package_name)
client.upload_pipeline_version(package_name, upload_pipeline.__name__)

Uploaded new pipeline
